In [1]:
from confluent_kafka import Producer

In [5]:
producer = Producer(conf)

%3|1726662685.051|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 303ms in state SSL_HANDSHAKE)
%3|1726662685.991|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 286ms in state SSL_HANDSHAKE, 1 identical er

In [7]:
topic = 'trade_data'

%3|1726662746.743|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 15ms in state SSL_HANDSHAKE, 30 identical error(s) suppressed)


In [8]:
producer.produce(topic, key="key1", value="Hello, Kafka from Python!", callback=delivery_report)

%3|1726662777.734|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 17ms in state SSL_HANDSHAKE, 31 identical error(s) suppressed)


In [ ]:
producer.flush()

%3|1726662807.735|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 13ms in state SSL_HANDSHAKE, 30 identical error(s) suppressed)
%3|1726662837.746|FAIL|my-python-producer#producer-1| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 19ms in sta

In [12]:
import os, time
from influxdb_client import InfluxDBClient, Point
import random
from kafka import KafkaProducer, KafkaConsumer
import json
import pandas as pd
import json
import datetime as dt
from time import sleep
from confluent_kafka import Producer

In [13]:
token = "pNa6-UfsFJUYd--ryyzXGDUk1o7Ei9BR1TKqO43WJ50uCmuJ30WDAHXk47Nc6vK8wALkJWejAJg09f-0nOE3cg=="
org = "Devnullx"
host = "http://165.232.189.85:8086/"
bucket = "trade_sim2" 
client = InfluxDBClient(url=host, token=token, org=org)

In [14]:
query = '''from(bucket: "trade_sim2")
  |> range(start: -1m)
  |> filter(fn: (r) => r["_measurement"] == "trade")
  |> filter(fn: (r) => r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low" or r["_field"] == "Open" or r["_field"] == "Volume" or r["_field"] == "buy_money" or r["_field"] == "buy_quant" or r["_field"] == "money_earned_user" or r["_field"] == "money_earned_platform" or r["_field"] == "open_time" or r["_field"] == "sell_money" or r["_field"] == "sell_quant" or r["_field"] == "total_money_invested" or r["_field"] == "user_loser" or r["_field"] == "user_profitable")
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")'''

In [15]:
query_api = client.query_api()

In [16]:
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

In [17]:
conf = {
    'bootstrap.servers': 'kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'doadmin',
    'sasl.password': 'AVNS_UObiidzUSi5rq9l-twR',
    'ssl.ca.location': '/etc/ssl/certs/ca-certificates.crt',  # This is the default CA cert location on most systems
    'client.id': 'my-python-producer'
}

In [19]:
topic = 'trade_data'

In [ ]:
while True:
    tables = query_api.query(query=query, org=org)
    records = []
    for table in tables:
        for record in table.records:
            records.append({
                'time': record.get_time(),
                '_value': record.get_value(),
                '_field': record.get_field(),
                '_measurement': record.get_measurement(),
                'tags': record.values  # Extract tag values if needed
            })

    df = pd.DataFrame(records)
    df = df.sort_values(by="time")

    df = df.drop(["time"],axis = 1)
    producer = Producer(conf)
    key = str(0).encode()  # Here, '0' is just an example; it can be any unique identifier
    df_dict = df.to_dict()
    data = json.dumps(df_dict, default=str).encode('utf-8')
    producer.produce(topic, key=key, value=data, callback=delivery_report)
    producer.flush()
    #producer.send(topic="trade_data", key=key, value=data)
    print(f'Sent record to topic at time {dt.datetime.utcnow()}')
    sleep(60)

%3|1726663433.941|FAIL|my-python-producer#producer-2| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 12ms in state SSL_HANDSHAKE)
%3|1726663434.940|FAIL|my-python-producer#producer-2| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 13ms in state SSL_HANDSHAKE, 1 identical erro

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Sent record to topic at time 2024-09-18 12:48:53.979759


%3|1726663734.995|FAIL|my-python-producer#producer-2| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 11ms in state SSL_HANDSHAKE, 31 identical error(s) suppressed)
%3|1726663765.002|FAIL|my-python-producer#producer-2| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 12ms in sta

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Sent record to topic at time 2024-09-18 12:54:54.107617


%3|1726664097.123|FAIL|my-python-producer#producer-3| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 14ms in state SSL_HANDSHAKE, 30 identical error(s) suppressed)
%3|1726664127.127|FAIL|my-python-producer#producer-3| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 12ms in sta

...

%3|1726669857.147|FAIL|my-python-producer#producer-19| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 9ms in state SSL_HANDSHAKE, 31 identical error(s) suppressed)
%3|1726669887.162|FAIL|my-python-producer#producer-19| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 11ms in st

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Sent record to topic at time 2024-09-18 14:36:56.247054


%3|1726670217.259|FAIL|my-python-producer#producer-20| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 9ms in state SSL_HANDSHAKE, 31 identical error(s) suppressed)
%3|1726670247.263|FAIL|my-python-producer#producer-20| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 9ms in sta

Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
Sent record to topic at time 2024-09-18 14:42:56.369909


%3|1726670577.380|FAIL|my-python-producer#producer-21| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 8ms in state SSL_HANDSHAKE, 31 identical error(s) suppressed)
%3|1726670607.389|FAIL|my-python-producer#producer-21| [thrd:sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocea]: sasl_ssl://kafka-database-do-user-17641554-0.h.db.ondigitalocean.com:25073/bootstrap: SSL handshake failed: error:0A000086:SSL routines::certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (install ca-certificates package) (after 11ms in st